In [1]:
!pip install pandarallel
!pip install wordninja

  Created wheel for pandarallel: filename=pandarallel-1.4.8-py3-none-any.whl size=16111 sha256=00e64288e5a5710c0dca76a014f410607f340abb69208c137f0395d5975996ef
  Stored in directory: /root/.cache/pip/wheels/c7/f2/4e/e40c8b9344cccf6b8a02d8d8808ba837e72b607c4be946878a
Successfully built pandarallel
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 541 kB 2.7 MB/s eta 0:00:01
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541552 sha256=f5716dc2b3d57e21ce8f73df25f506c23402dc6626c2a482a4553961c45a456b
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import wordninja
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import nltk
import pickle
import heapq
from collections import OrderedDict
import pandarallel
from gensim import corpora, models
from gensim.utils import lemmatize
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
import ast
from tqdm import tqdm
import gensim
import pickle
from gensim.models import CoherenceModel, LdaMulticore, TfidfModel
tqdm.pandas()
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from textblob import TextBlob, Word

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Data Cleaning

# kept only noun, adjective, verb, adverb
# lemmatize using POS tag
# tried different no. of topics to find the final num of topics using coherence
# batch wise LDA also done

In [3]:
## intermediate cleaning

def removal(text):
    text = re.sub('XXXX',' UNKNOWN ',text)
    text = re.sub('XX/XX/','',text)
    text = re.sub('UNKNOWN   UNKNOWN','UNKNOWN',text)
    text = re.sub('\n',' ',text)
    text = re.sub(r'\s{2,}',' ',text)
    return text

def lemmatize_with_postag(sentence):
    text = removal(sentence)
    text = text.lower()
    text = re.sub(r'(unknown\s)+',' ',text)
    sent = TextBlob(text)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags if pos in ['JJ','JJR','JJS','NNS','NNP','NNPS','NN','RB','RBR','RBS','VB','VBG','VBD','VBN','VBP','VBZ']] 
    lemmatized_sent = ''.join([wd.lemmatize(tag) for wd, tag in words_and_tags if wd not in stopwords.words('english')])
    clean_sent = " ".join(wordninja.split(lemmatized_sent))
    return clean_sent

# code for parallel processing in pandas

In [4]:
pandarallel.pandarallel.initialize(progress_bar= True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# training and test data code for gensim LDA
Gensim requires list of list of tokens for all sentences in the corpus and a dictionary for mapping all vocabulary to BOW. I have already cleaned the data using the cleaning code.

In [5]:
complaints = pd.read_csv('/kaggle/input/lda-top500-words/top500tfidfwords(1).csv',encoding= 'unicode_escape')
complaints.dropna(subset=['Consumer complaint narrative'],inplace =True)
complaints.drop_duplicates(subset=['Consumer complaint narrative'],inplace =True)

# list_of_list_of_tokens = [["a","b","c"], ["d","e","f"]]
# ["a","b","c"] are the tokens of document 1, ["d","e","f"] are the tokens of document 2...

In [6]:
# %%time
# list_of_list_of_tokens = complaints['Consumer complaint narrative'].parallel_apply(word_tokenize)
# list_of_list_of_tokens = list_of_list_of_tokens.to_list()

CPU times: user 3.47 s, sys: 1.26 s, total: 4.72 s
Wall time: 36.1 s


In [7]:
len(list_of_list_of_tokens),complaints.shape

(134289, (134289, 6))

In [8]:
%%time
test_data = pd.read_csv('/kaggle/input/test-clean-data/test_clean.csv', encoding= 'unicode_escape')

# test_tokens = test_data['Clean_narrative'].parallel_apply(word_tokenize)

# test_tokens = test_tokens.to_list()


CPU times: user 332 ms, sys: 180 ms, total: 512 ms
Wall time: 1.42 s


In [ ]:
## do not run this cell we do not want to remove any row
# test_data.dropna(subset=['Clean_narrative'],inplace =True)
# test_data.drop_duplicates(subset=['Clean_narrative'],inplace =True)

In [9]:
# test_data.shape,len(test_tokens)

((4282, 2), 4282)

In [ ]:
# if you have saved list of tokens to file, when reading next time use this
# clean_df['list_of_list_of_tokens']= clean_df['list_of_list_of_tokens'].parallel_apply(ast.literal_eval)

# code for making vocab dictionary for BOW & saving it for later use

In [ ]:
# dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
# filename = 'dictionary_LDA_15mon.sav'
# pickle.dump(dictionary_LDA, open(filename, 'wb'))

# load saved dictionary

In [10]:
# pickle_in = open('/kaggle/input/lda-dictionary-15mon/dictionary_LDA_15mon.sav',"rb")
# dictionary_LDA = pickle.load(pickle_in)

# convert docs to BOW, this is the proper format gensim LDA's expects the data to be 

In [11]:
# dictionary_LDA.filter_extremes(no_below=3)
# corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

# Fitting the model on doc2bow, printing top 50 words, saving the model & visualising

In [ ]:
%%time

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
                    
# # Model by num_products
# num_topics = 8
# lda_model = models.LdaMulticore(corpus, num_topics=num_topics, \
#                                   id2word=dictionary_LDA, \
#                                   passes=3, alpha=0.01*num_topics, \
#                                   eta=0.01*len(dictionary_LDA.keys()))

# for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=50):
#     print(str(i)+": "+ topic)
#     print()
    
# filename = 'lda_model_28th_May.sav'
# pickle.dump(lda_model, open(filename, 'wb'))

# from gensim.models import CoherenceModel# Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=list_of_list_of_tokens, dictionary=dictionary_LDA, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus_tfidf, dictionary=lda_model.id2word)
# vis

# following piece of code can be used for Gensim's LDA model evaluation

In [ ]:
# def compute_coherence_values(corpus, dictionary, k, a, e):
    
#     lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                            id2word=dictionary,
#                                            num_topics=k, 
#                                            random_state=100,
#                                            passes=3,
#                                            alpha=a,
#                                            eta=e,
#                                            per_word_topics=True)
    
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=test_tokens, dictionary= dictionary, coherence='c_v')
    
#     return coherence_model_lda.get_coherence()

# training and test corpus for sklearn's LDA

## creating BOW with count vectorizer, both sklearn & gensim LDA work great with cv BOW
the below two line of code are good to go you can use the result of count vectorizer as input to your LDA model

In [12]:
# cv = CountVectorizer()
# data_vectorized = cv.fit_transform(complaints['Consumer complaint narrative'])
# test_data_vectorized = cv.transform(test_data['Clean_narrative'])



In [36]:
complete_data = pd.concat([complaints['Consumer complaint narrative'],test_data['Clean_narrative']])

cv = CountVectorizer()
data_vectorized = cv.fit_transform(complete_data)

complete_data.shape,complaints.shape, test_data.shape

((138554,), (134272, 6), (4282, 3))

# TF-IDF doesn't work well with sklearn, gensim LDA but we can use top n words from each complaint, you can go for n-grams as well later on

In [13]:
tf = TfidfTransformer()
tfidf_transformed_train_data = tf.fit_transform(data_vectorized)
# tfidf_transformed_test_data = tf.transform(test_data_vectorized)
feature_names=cv.get_feature_names()

In [16]:
sum=0
words = set(nltk.corpus.words.words())
def extract_topn_from_vector(doc,topn=500):
    global sum, words
    doc = " ".join([w for w in nltk.word_tokenize(doc) if len(w)>1 and (w in words or w.isalnum())])
    tf_idf_vector=tf.transform(cv.transform([doc]))
    coo_matrix = tf_idf_vector.tocoo()
    
    top500=heapq.nlargest(topn, coo_matrix.data)
    
    dictionary = OrderedDict(dict())
    
    for idx,score in list(zip(coo_matrix.col,coo_matrix.data)):
        dictionary[feature_names[idx]]=score
    
    words = [w for w in doc.split() if w not in stopwords.words('english')]
    try:        
        results = OrderedDict({x:dictionary[x] for x in words if dictionary[x] in top500})
        return ' '.join(results.keys())
    except:
        sum+=1
        print(doc,'not converted',sum)
        return doc

In [27]:
# testing extract_topn_from_vector finction
x=9
test_data['Clean_narrative'][x],extract_topn_from_vector(test_data['Clean_narrative'][x])

('receive call student loan company state debt owe never apply student loan never go college quickly open account review credit report find student loan fraudulent account never open give anyone consent open please delete item credit file soon possible',
 'receive call student loan company state debt owe never apply go college quickly open account review credit report find fraudulent give anyone consent please delete item file soon possible')

In [28]:
# test_data['top500'] = test_data['Clean_narrative'].parallel_apply(extract_topn_from_vector)

today date attention dispute department professional debt company violate fcra prohibition sale transfer debt cause identity theft respond request provide investigation report need resolve fair credit reporting act dispute item appear credit report support evidence general person company sell transfer consideration place collection debt person notify section title result identity theft applicability prohibition subsection apply person collect debt describe paragraph date notification paragraph rule construction nothing subsection construe prohibit repurchase debt case assignee debt require repurchase debt result identity theft securitization debt pledging portfolio debt collateral connection borrowing transfer debt result merger acquisition purchase assumption transaction transfer substantially asset entity person furnish information consumer reporting agency place reasonable procedure respond notification receive consumer reporting agenc section title relate information result identit

recently open complaint cfpb regard navy federal credit union discriminate veteran cfpb complaint dated still due back answer however recently receive ironic demean undermine email representative complain attach new cfpb complaint well audacity send email attempt help still state base decision nfcu equity credit line credit score lie organization internal bias credit score rating also deny intentionally block homeowner receive equity home line credit home block economic financial resource also form racial discrimination apply equity line credit reference cfpb complaint dated inform mortgage loan manager score need least qualify equity home line credit credit score still scoring parameter discuss however receive letter nfcu deny regard internal discriminatory scoring system also identify applicant well also inform escalate send letter relate extenuate circumstance request credit exception complete send referenced attach cfpb complaint dated ignore request surely content letter take cons

dispute credit company account settle pay full reach agreement pay amount exerpian report make still owe dispute info also submit email state pay full still settle debt company company dispute well keep account open relocate electric account switch energy submit proof relocate state still side not converted 3
concern reason dispute allied inerstate llc acct send debt collection letter recognize dispute section subsection however record unable locate documentation comply requirement violation fcra fdcpa also ally inerstate llc fail give day prior place derogatory item credit report section validation date section807 ally inerstate llc fail send initial communication place allege item credit report violate fcra violation ally inerstate llc hold liable well defamation state ohio indebtedness creditor individual debtor codebtors fdcpa fail see allied inerstate llc compliance ohio law also report account ftc police need collection account remove credit inerstate llc even try contact regard 

president trump sign coronavirus stimulas bill know exactly wording bill state however something credit score negatively affect due pandemic happen crisis hugely affect get credit card small limit help time new credit report come credit score drop point understand credit reporting agency feel power god people report credit lender fight tooth nail year bring credit score seem make headway even past month never miss payment anything let alone late however get score past drop point get new card crisis criminal offense not converted 5
loancare help place customer covid time advertise front page thers help forebearance demand full payment owe amount period forbearance help work mandortary shut state government dont income able pay month bank offer deferment helpful make sense someone please help dont want lose home due epidemic already call avail not converted 6
education still report credit report unpaid account pay full report pay full credit reporting agency originally request original c

well fargo bank open unauthorized checking savings account flag account fraud identity theft well fargo close account approval well fargo wf charge account penalty cause account negative wf start series overdraft fee recur deepen negative balance wf send account collection wf refuse give account information account go back simply contact initial opening branch confirm access account direct violation fcra intentionally commit various form fraud intentionally cause economic harm wf promise assign caseworker call fail call harm lock credit report apply sba ppp loan federal care act close checking account wf serve letter senator massachusetts answer fraud http well fargo commit several violation federal banking consumer saving credit law wf violate california privacy right well fargo bank pay cover fraud still close people account daily give access record ticker symbol well fargo bank wfc trade new york stock exchange dollar asset large bank united state well fargo shortable stock market u

...

transunuon continue report fraudulent complaint fill transunion back report payment current month transunion report late payment not converted 92
back account delete reinsurted new account violation fcra rule please see attachment believe original creditor balance reinserted intentionally hurt damage credit score profile violation fcra rule regulation fdcpa not converted 83
change amount less amount myloancares website withdraw checking withdraw saving ive issue month straight dont know else ive incur worth fee company bank quickly realize wasnt fault myloancare email request refund isnt fault myloancare take money correct bank try take money saving even check not converted 93
send payment single month company sl figure process send payment back change payment history day late month give refund payment take day ago bank record confirm send time post monthly payment late accounting probelms not converted 84
write credit bureau address error find result dispute information come back veri

sign loan document license owner contact superior loan service representative name clarify question manager question clarify see note loan term ask default interest make clear interest charge due legal notice default file closing pad final seller statement addition th interest add interest line item inflated amount equivalent double amount due closing misrepresent charge regular interest value default interest never default ever receive notice default legal notice default ever file ask payment say default interest never receive default notice itemized line item specify exactly charge receive multiple harass phone call email servicer superior loan servicing mortgage loan originator state default duress servicer constantly call threaten take house foreclosure send email present coercive term investor pay loan owe take house keep say otherwise sue afford attorney tell contract require notice default never receive default mortgage loan originator servicer know wait investor buy house priva

debt income ratio work collect retirement accurately evaluate credibility try report issue experian let dispute issue even year past still wont let dispute process issue make say something else order look make sense especially inaccuracy fix even allow look account ti boost credit yet inaccuracy credit report send inaccurate information second issue home negative score own home year sell different state year later credit make difficult purchase fair recent retired veteran still income come inaccurately elevate credit score personal never pay bill case collection identity theft happen hospitalize provide information support company keep report debt credit happen year month date happen seem fresh issue illegal false advertising false information collection current date happen already prove fraud still know company negative whole report year work bankruptcy collection issue work follow debt work due unemployment reason mean nearly die defend country work time recover try make right identi

several credit card bank account loan apply use personal information give consent anyone use information account open fraudulentlyi write request initiate formal complaint experian cra mail experian block fraudulent information report account victim identity theft attach unwilling unable conduct adequate investigation fail refuse block usage fraudulent account report please contact agency initiate formal investigation possible violation fair credit reporting act fair debt collection practice act also contact attorney general postal inspection service possible violation law not converted 102
ongoing late trouble pay minimum payment high interest high minimum payment credit card call ask anything get monthly payment hard time financially divorce afford monthly bill take care kid keep get late fee added interest even payment day late never day reduce interest rate say payment plan reduce payment month go back normal rate agree read fine print report credit report close credit card account

terminate lease early local apartment complex accept surrender apartment see attach email trigger duty mitigate damage applicable georgia law accordingly cant require repay entire term lease lease property someone else see additional email communication confirm relent apartment process determine final amount owe property however never receive final account statement tell owe see email attachment receive notice iq data international attempt collect behalf apartment complex never even issue final bill complex much less wind collection ask explanation iq data representative spoke derogatory manner tell owe full amount accelerated rent tell lawfully possible georgie law never even receive final bill complex accept surrender property relet contact ga refuse send copy final account statement allegedly send instead send screen shot supposedly email copy final statement request resend original email final account statement receive response iq data increase amount owe speak today employee say l

In [ ]:
# complaints.to_csv('top500tfidfwords.csv')

In [ ]:
# complaints['original_len'] = complaints['Consumer complaint narrative'].str.len()
# complaints['top500_len']= complaints['top500'].str.len()
complaints.describe()

In [29]:
complaints.dropna(subset=['top500'], inplace=True)
cv_top500 = CountVectorizer()
top500_words_vector = cv_top500.fit_transform(complaints['top500'])
top500_test_vector = cv_top500.transform(test_data['top500'])

# code for model tuning for sklearn's LDA

In [ ]:
# from sklearn.decomposition import LatentDirichletAllocation
# from sklearn.model_selection import GridSearchCV

# search_params = {'n_components': [4,8,10,16], 'learning_decay': [.7]}
# # search_params = {'n_components': [10,15,20], 'learning_decay': [.5, .7, .9]}

# # Init the Model
# lda = LatentDirichletAllocation()

# # Init Grid Search Class
# model = GridSearchCV(lda, param_grid=search_params)

# # Do the Grid Search
# model.fit(top500_words_vector)

In [ ]:
# best_lda_model = model.best_estimator_

# # Model Parameters
# print("Best Model's Params: ", model.best_params_)

# # Log Likelihood Score
# print("Best Log Likelihood Score: ", model.best_score_)

# # Perplexity
# print("Model Perplexity: ", best_lda_model.perplexity(top500_words_vector))

In [ ]:
# # Get Log Likelyhoods from Grid Search Output
# # n_topics = [10,15,20]
# n_topics = [4,8,10,16]
# # log_likelyhoods_5 = [round(model.cv_results_['mean_test_score'][index]) for index, gscore in enumerate(model.cv_results_['params']) if gscore['learning_decay']==0.5]
# log_likelyhoods_7 = [round(model.cv_results_['mean_test_score'][index]) for index, gscore in enumerate(model.cv_results_['params']) if gscore['learning_decay']==0.7]
# # log_likelyhoods_9 = [round(model.cv_results_['mean_test_score'][index]) for index, gscore in enumerate(model.cv_results_['params']) if gscore['learning_decay']==0.9]

# # Show graph
# plt.figure(figsize=(12, 8))
# # plt.plot(n_topics, log_likelyhoods_5, label='0.5')
# # plt.plot(n_topics, log_likelyhoods_7, label='0.7')
# plt.plot(n_topics, log_likelyhoods_9, label='0.9')
# plt.title("Choosing Optimal LDA Model")
# plt.xlabel("Num Topics")
# plt.ylabel("Log Likelyhood Scores")
# plt.legend(title='Learning decay', loc='best')
# plt.show()

# LDA sklearn fit

sklearn LDA expects BOW only using tfidf messed up the model

In [ ]:
# # # Load the LDA model from sk-learn
# # from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# # Helper function
# def print_topics(model, count_vectorizer, n_top_words):
#     words = count_vectorizer.get_feature_names()
#     for topic_idx, topic in enumerate(model.components_):
#         print("\nTopic #%d:" % topic_idx)
#         print(" ".join([words[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))

# model with 6 topics

In [ ]:
# # Tweak the two parameters below
# number_topics = 6
# number_words = 20
# learning_decay=0.9


# # Create and fit the LDA model
# lda6 = LDA(n_components=number_topics, n_jobs=-1,learning_decay=learning_decay)
# lda6.fit(top500_words_vector)# Print the topics found by the LDA model
# print("Topics found via LDA:")
# print_topics(lda6, cv_top500, number_words)
# # Perplexity
# print("Model Perplexity: ", lda6.perplexity(top500_words_vector))
# import pyLDAvis
# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()
# panel = pyLDAvis.sklearn.prepare(lda6, top500_words_vector, cv_top500, mds='tsne')
# panel

In [ ]:
# filename = 'LDA6.sav'
# pickle.dump(lda6, open(filename, 'wb'))

# model with 8 topics

In [ ]:
# # Tweak the two parameters below
# number_topics = 8
# number_words = 20
# learning_decay=0.9


# # Create and fit the LDA model
# lda8 = LDA(n_components=number_topics, n_jobs=-1,learning_decay=learning_decay)
# lda8.fit(top500_words_vector)# Print the topics found by the LDA model
# print("Topics found via LDA:")
# print_topics(lda8, cv_top500, number_words)
# # Perplexity
# print("Model Perplexity: ", lda8.perplexity(top500_words_vector))
# import pyLDAvis
# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()
# panel = pyLDAvis.sklearn.prepare(lda8, top500_words_vector, cv_top500, mds='tsne')
# panel

In [ ]:
# filename = 'LDA8.sav'
# pickle.dump(lda8, open(filename, 'wb'))

# model with 4 topics

In [ ]:
# # Tweak the two parameters below
# number_topics = 4
# number_words = 20
# learning_decay=0.9


# # Create and fit the LDA model
# lda4 = LDA(n_components=number_topics, n_jobs=-1,learning_decay=learning_decay)
# lda4.fit(top500_words_vector)# Print the topics found by the LDA model
# print("Topics found via LDA:")
# print_topics(lda4, cv_top500, number_words)
# # Perplexity
# print("Model Perplexity: ", lda4.perplexity(top500_words_vector))
# import pyLDAvis
# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()
# panel = pyLDAvis.sklearn.prepare(lda4, top500_words_vector, cv_top500, mds='tsne')
# panel

In [ ]:
# filename = 'LDA4.sav'
# pickle.dump(lda4, open(filename, 'wb'))

In [ ]:
# import pyLDAvis
# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()
# panel = pyLDAvis.sklearn.prepare(best_lda_model, top500_words_vector, cv_top500, mds='tsne')
# panel

# code for demo begins from here

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()
# data_vectorized = cv.fit_transform(complaints['Consumer complaint narrative'])
# test_data_vectorized = cv.transform(test_data['Consumer complaint narrative'])

In [30]:
import pickle
pickle_in = open('/kaggle/input/tfidf-lda-8/LDA8.sav',"rb")
lda8 = pickle.load(pickle_in)

# topic top 50 keywords

In [ ]:

# # Helper function
# number_words = 50
# topic_dict={}
# def print_topics(model, count_vectorizer, n_top_words):
#     words = count_vectorizer.get_feature_names()
#     for topic_idx, topic in enumerate(model.components_):
#         print("\nTopic #%d:" % topic_idx)
#         print(" ".join([words[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
#         topic_dict["\nTopic #%d:" % topic_idx] = " ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        

# print_topics(lda8, cv_top500, number_words)



# topics in test data

In [31]:

# Create Document - Topic Matrix
lda_output = lda8.transform(top500_test_vector)

# column names
topicnames = ["Topic" + str(i) for i in range(lda8.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(top500_test_vector.shape[0])]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
# df_document_topic.to_csv('LDA_test_data_result')
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.340000,0.010000,0.600000,0.010000,6
Doc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.570000,0.410000,0.000000,5
Doc2,0.010000,0.360000,0.600000,0.010000,0.010000,0.010000,0.010000,0.010000,2
Doc3,0.010000,0.340000,0.610000,0.010000,0.010000,0.010000,0.010000,0.010000,2
Doc4,0.010000,0.430000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,2
Doc5,0.000000,0.210000,0.000000,0.090000,0.000000,0.340000,0.360000,0.000000,6
Doc6,0.080000,0.000000,0.000000,0.520000,0.000000,0.400000,0.000000,0.000000,3
Doc7,0.000000,0.370000,0.000000,0.000000,0.060000,0.000000,0.000000,0.550000,7
Doc8,0.880000,0.000000,0.000000,0.000000,0.120000,0.000000,0.000000,0.000000,0
Doc9,0.000000,0.000000,0.530000,0.000000,0.290000,0.000000,0.160000,0.000000,2


array([[0.01042522, 0.01043732, 0.01042393, ..., 0.010421  , 0.59799546,
        0.0104254 ],
       [0.00313131, 0.00312941, 0.00313127, ..., 0.57093048, 0.41028699,
        0.00313181],
       [0.00659634, 0.36092367, 0.5995429 , ..., 0.00658302, 0.00658616,
        0.00659024],
       ...,
       [0.00417412, 0.00417146, 0.00417453, ..., 0.00417445, 0.00417274,
        0.6923557 ],
       [0.0021194 , 0.00211994, 0.00212155, ..., 0.30885086, 0.16203703,
        0.00212079],
       [0.01250873, 0.0125094 , 0.91242893, ..., 0.01250252, 0.0125081 ,
        0.01250806]])

# topic distribution in april test data

In [ ]:
# df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
# df_topic_distribution.columns = ['Topic Num', 'Num Documents']
# df_topic_distribution

In [ ]:
# df_document_topic['complaint'] = test_data['Clean_narrative'].values
# # df_document_topic['original_complaint'] = test_data['Consumer complaint narrative'].values

In [ ]:
# def add_topic_in_test(x):
#     if x == 0:
#         return topic_dict['\nTopic #0:']
#     elif x == 1:
#         return  topic_dict['\nTopic #1:']
#     elif x == 2:
#         return topic_dict['\nTopic #2:']
#     elif x == 3:
#         return topic_dict['\nTopic #3:']
#     elif x == 4:
#         return topic_dict['\nTopic #4:']
#     elif x == 5:
#         return topic_dict['\nTopic #5:']
#     elif x == 6:
#         return topic_dict['\nTopic #6:']
#     elif x == 7:
#         return topic_dict['\nTopic #7:']

In [ ]:
# df_document_topic['topic_keywords'] = df_document_topic['dominant_topic'].parallel_apply(add_topic_in_test)
# df_document_topic.to_csv('final_april_test_results.csv')

# topic distribution in training data

In [ ]:
# # Create Document - Topic Matrix
# lda_output = lda8.transform(top500_words_vector)

# # column names
# topicnames = ["Topic" + str(i) for i in range(lda8.n_components)]

# # index names
# docnames = ["Doc" + str(i) for i in range(top500_words_vector.shape[0])]

# # Make the pandas dataframe
# df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# # Get dominant topic for each document
# dominant_topic = np.argmax(df_document_topic.values, axis=1)
# df_document_topic['dominant_topic'] = dominant_topic

# df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
# df_topic_distribution.columns = ['Topic Num', 'Num Documents']
# df_topic_distribution

In [ ]:
# import pyLDAvis
# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()
# panel = pyLDAvis.sklearn.prepare(best_lda_model, test_data_vectorized, cv, mds='tsne')
# panel